In [9]:

from typing import Annotated,Sequence,TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage,HumanMessage,ToolMessage,BaseMessage
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from langchain.tools import tool
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv

In [10]:
load_dotenv()

True

In [11]:

class Agentstate(TypedDict):
    message:Annotated[Sequence[BaseMessage],add_messages]

In [12]:
@tool
def addition(a:int,b:int):
    """this is for addition of two number"""
    return a + b

@tool
def multiply(a:int,b:int):
    """this is the for multiplication"""
    return a*b

tools = [addition,multiply]
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash').bind_tools(tools)


In [13]:
def model_call(state:Agentstate):
    system_prompt = SystemMessage(content="you can well qualify doctor.")
    response = llm.invoke([system_prompt] + state["message"])
    return {'message':response.content}

def should_continue(state:Agentstate):
    """This is for the condition based means for looping using tools or not."""
    message_1 = state["message"]
    last_message = message_1[-1]
    if last_message.tool_calls:
        return "end"
    else:
        return "continue"
    


In [ ]:
graph = StateGraph(Agentstate)
graph.add_node("our_agent",model_call)
tool_node = ToolNode(tools=tools)
graph.set_entry_point("our_agent")
graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "continue":"tools",
        "end":END
    }
)